# EDA
---
### We begin by importing the dataset. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Put each csv into a dataframe. 

In [ ]:
mouse_data = pd.read_csv('/kaggle/input/pro-overwatch-player-settings/mouse_data.csv')
owl_roster = pd.read_csv('/kaggle/input/pro-overwatch-player-settings/owl_roster.csv')
owl_player_settings = pd.read_csv('/kaggle/input/pro-overwatch-player-settings/player_settings.csv')
owl_player_settings.head()

### Identify and remove outliers for each role. 

In [ ]:
sns.stripplot(x='Role', y='eDPI', data=owl_player_settings)

In [ ]:

# Split dataset into three separate dataframes grouped by roles
tanks = owl_player_settings[owl_player_settings["Role"] == "Tank"]
supps = owl_player_settings[owl_player_settings["Role"] == "Support"]
dmgs = owl_player_settings[owl_player_settings["Role"] == "Damage"]

# Calculate low 1% and high 99% quantile values for each role
q_tank_high = tanks.quantile(0.99)["eDPI"]
q_tank_low = tanks.quantile(0.01)["eDPI"]

q_supp_high = supps.quantile(0.99)["eDPI"]
q_supp_low = supps.quantile(0.01)["eDPI"]

q_dmg_high = dmgs.quantile(0.99)["eDPI"]
q_dmg_low = dmgs.quantile(0.01)["eDPI"]

# Remove outliers from each dataset role
tanks = tanks[(tanks["eDPI"] > q_tank_low) & (tanks["eDPI"] < q_tank_high)]
supps = supps[(supps["eDPI"] > q_supp_low) & (supps["eDPI"] < q_supp_high)]
dmgs = dmgs[(dmgs["eDPI"] > q_dmg_low) & (dmgs["eDPI"] < q_dmg_high)]


# Combine all dataframes and update the original dataframe
all_df = [tanks, supps, dmgs]
owl_player_settings = pd.concat(all_df)

### Calculate average eDPI for each role.

In [ ]:
# Calculate averages based on roles
average_by_role = owl_player_settings.groupby("Role").mean()

edpi_tank = str(round(average_by_role['eDPI']['Tank'], 2))
edpi_supp = str(round(average_by_role['eDPI']['Support'], 2))
edpi_dmg = str(round(average_by_role['eDPI']['Damage'], 2))

sns.barplot(x='Role', y='eDPI', data=owl_player_settings)

# Analysis
The difference in average eDPI, to any competitive Overwatch player or viewer, makes sense - and here's why. You can calculate how many centimeters (cm) of mouse movement you need to do a 360 degree turn in game by diving a large constant by the eDPI (eDPI and cm/360 are inversely correlated). This means that the the more you raise your eDPI, the less you need to move your mouse physically to perform a full 360 in game. The advantage of a high eDPI is that you can turn faster in-game with less physical movement, granting you more speed and flexibility. However, the disadvantage is that your physical mouse movements need to be more accurate. If a player using a high eDPI moves their mouse 5cm too far to the left, it will look much worse in-game than if a low eDPI player did.

Most players choose their in-game sensitivity with the balance between flexibility and accuracy in mind as well as their in-game playstyle and duties. Others just go by what feels natural. Either way, these players are professionals so they know what they're doing!  

That said, here are the explanations for the average eDPI of each role. 

Reminder that these are all **general**. There are many exceptions mainly due to the versatility of heros/playstyles in each role and, of course, the fact that sensitivity ultimately comes down to personal prefernece. 

* **Tanks**: In general, tank heroes require less accuracy than other roles because of thier in-game duties. This explains the higher average eDPI. On average, tank players will sacrifice accuracy for flexibility in their aim. However, the variance is also high because some tank heros focus on protecting the team and creating positional opportunities while other tank heros focus on helping the other tank do their job while dealing damage to opponents. Becasue heroes with vastly different playstyles are categorized into one category as "tank", the variance is high. 

* **Supports**: Similar to the tank role, heroes in the support category have a variety of different playstyles, which explains the high variance. However, on average, the eDPI is the lowest of the three caregories. This is most likley due to certain support heroes needing extremely precise aim. Their roles include healing the team and also dealing damage. Both actions require accurate aim, thus, bringing the average eDPI down. However, some support heroes require little to no aim which brings the eDPI and variance up. 

* **Damage**: The damage role has the smallest variance and a bit higher average eDPI than the support role. The main role of all damage heroes is obviously to deal damage. Unlike  other roles, the damage role better categorizes the heroes because they all have the same duty: to deal damage. This explains the low variance. The average sensitivity, however, is slightly higher than supports. This is most likely because, although most damage heroes have the same duty, they all perform their duty in different ways. Some heroes have sniper rifles which require pin-point accuracy, while others have projectile weapons which have splash damage (damages a general area) and do not need to be that accurate. The accuracy needed in order to use these different weapons vary. This is where the small bit of variance as well as the slightly high average eDPI comes from.

In [ ]:
# Calculate averages based on heroes
average_by_hero = owl_player_settings.groupby("Hero").mean()
sns.set(rc={'figure.figsize':(25,9)})
sns.barplot(x='Hero', y='eDPI', data=owl_player_settings.sort_values("cmPer360"))

In [ ]:
# Reference: https://www.kaggle.com/joshuaswords/awesome-eda-2021-happiness-population

QUARTZ = '#4A4C4E'
YELLOW_ORANGE = '#FA9C1D'
WHITE = '#FFF'

fig=plt.figure(figsize=(7,3),facecolor=WHITE)

ax0=fig.add_subplot(1,1,1)
ax0.text(0.50,0.7,"Average eDPI",color=QUARTZ,fontsize=35, fontweight='bold', fontfamily='calibri',ha='center')

ax0.text(0,0.4, edpi_tank,color=YELLOW_ORANGE,fontsize=25, fontweight='bold', fontfamily='calibri',ha='center')
ax0.text(0,0.2,"Tank",color=QUARTZ,fontsize=20, fontweight='light', fontfamily='calibri',ha='center')

ax0.text(1,0.4,edpi_dmg,color=YELLOW_ORANGE,fontsize=25, fontweight='bold', fontfamily='calibri',ha='center')
ax0.text(1,0.2,"Damage",color=QUARTZ,fontsize=20, fontweight='light', fontfamily='calibri',ha='center')

ax0.text(0.5,0.4,edpi_supp,color=YELLOW_ORANGE,fontsize=25, fontweight='bold', fontfamily='calibri',ha='center')
ax0.text(0.5,0.2,"Support",color=QUARTZ,fontsize=20, fontweight='light', fontfamily='v',ha='center')


ax0.set_yticklabels('')
ax0.tick_params(axis='y',length=0)
ax0.tick_params(axis='x',length=0)
ax0.set_xticklabels('')
ax0.set_facecolor(WHITE)
for direction in ['top','right','left','bottom']:
    ax0.spines[direction].set_visible(False)

# Looking at Mouse Data

In [ ]:
top_5_mice = dict(owl_player_settings['MouseModel'].value_counts()[:5])
fig=plt.figure(figsize=(20,5),facecolor=WHITE)

plt.title("Most Popular Mice")
plt.bar(range(len(top_5_mice)), list(top_5_mice.values()), align='center')
plt.xticks(range(len(top_5_mice)), list(top_5_mice.keys()))
plt.show()

Currently, Logitech seems to be the king in OWL in terms of mice. Perhaps there is a bias due to sponsorships? Or maybe because thier wirless mice are perceived to be more reliable than other brands? Nevertheless, Logitech makes some awesome mice so I don't blame anyone for using them! 